### 목표
- 네이버 영화리뷰 데이터셋을 이용해 긍정/부정 분류기를 만들어보자(0이면 부정, 1이면 긍정)
- TF-IDF (단어의 빈도와 문서내의 빈도를 같이 고려) 방법을 적용해보자
- Konlpy(한국어 형태소 분석기)를 설치하고 활용해보자
- 단어별 긍정/부정 정보를 확인해보자

In [1]:
!pip install konlpy

  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.9.3
    Uninstalling beautifulsoup4-4.9.3:
      Successfully uninstalled beautifulsoup4-4.9.3


### 네이버 영화리뷰 데이터 로딩

In [2]:
import pandas as pd
# set_option : 데이터 프레임의 환경을 설정해주는 명령
# display.max_colwidth : 최대 열 너비를 설정(None으로 설정시 무제한)
pd.set_option('display.max_colwidth', None)
# 영화 리뷰는 길기 때문에 모두 볼 수 있게끔 데이터프레임의 열 너비를 설정

In [6]:
df_train = pd.read_csv('data/ratings_train.txt', delimiter='\t')
df_test = pd.read_csv('data/ratings_test.txt', delimiter='\t')
# delimiter : 파일에서 데이터 사이의 간격을 구분해 줄 구분자를 설정하는 명령
# 별도의 설정을 하지 않는 경우에는 기본 csv 파일처럼 ,로 구분함

In [7]:
df_train.head(5)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다,1


In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [9]:
# dropna : 결측치(null값)를 하나라도 포함하는 행을 제거
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

In [10]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [11]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


In [12]:
text_train = df_train['document']
y_train = df_train['label']
text_test = df_test['document']
y_test = df_test['label']

In [14]:
print(text_train.shape)
print(y_train.shape)
print(text_test.shape)
print(y_test.shape)

(149995,)
(149995,)
(49997,)
(49997,)


### Bow 기법(1. 카운트 벡터라이저, 2. TF-IDF)
- 1. 카운트 벡터라이저 : 단순히 단어의 빈도수만 파악
- 2. TF-IDF : 단어의 빈도수 + 단어가 등장하는 문서의 개수 모두 파악

In [15]:
# TF-IDF 방식을 사용하기 위해 Tfidf 벡터라이저를 임포트
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
# Tfidf 벡터라이저 로드
tmp_tf_idf = TfidfVectorizer()

In [17]:
# 3개의 행까지만 단어가방으로 만들어보자
# fit : bow(단어가방)으로 만들어주기
tmp_tf_idf.fit(text_train[:3])

TfidfVectorizer()

In [18]:
# vocabulary_ : 토큰화된 단어와 매핑된 인덱스를 확인
tmp_tf_idf.vocabulary_

{'더빙': 2,
 '진짜': 6,
 '짜증나네요': 7,
 '목소리': 3,
 '포스터보고': 9,
 '초딩영화줄': 8,
 '오버연기조차': 5,
 '가볍지': 0,
 '않구나': 4,
 '너무재밓었다그래서보는것을추천한다': 1}

In [19]:
# 토큰화된 단어만 보기
tmp_tf_idf.get_feature_names()

['가볍지',
 '너무재밓었다그래서보는것을추천한다',
 '더빙',
 '목소리',
 '않구나',
 '오버연기조차',
 '진짜',
 '짜증나네요',
 '초딩영화줄',
 '포스터보고']

- 토큰화가 제대로 이루어지지 않고 있음

### konlpy - kkma 명령 사용해보기
- konlpy : 한글을 형태소마다 잘 토큰화 시킬 수 있는 도구들의 집합
- kkma : konlpy 중 가장 많이 사용하는 도구

In [20]:
from konlpy.tag import Kkma

In [21]:
kkma = Kkma()

SystemError: java.nio.file.InvalidPathException: Illegal char <*> at index 55: C:\Users\vdi02\Anaconda3\Lib\site-packages\konlpy\java\*